<a href="https://colab.research.google.com/github/Rizvi999/neural-lab/blob/main/11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import requests

# -------------------------------
# 1. Device setup
# -------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# -------------------------------
# 2. Download and preprocess text
# -------------------------------
url = 'https://www.gutenberg.org/files/1342/1342-0.txt'  # Pride and Prejudice
response = requests.get(url)
text = response.text

# Preprocess text: lowercase, only letters and space
text = text.lower()
text = ''.join(c for c in text if c.isalpha() or c.isspace())

# Reduce corpus size for faster training
max_chars = 100000
text = text[:max_chars]

# Character mappings
chars = sorted(list(set(text)))
char2idx = {ch: idx for idx, ch in enumerate(chars)}
idx2char = {idx: ch for idx, ch in enumerate(chars)}
vocab_size = len(chars)
print(f"Vocabulary size: {vocab_size}")

# Convert text to integers
text_as_int = np.array([char2idx[c] for c in text], dtype=np.int64)

# -------------------------------
# 3. Define LSTM model
# -------------------------------
class CharLSTM(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers=1):
        super(CharLSTM, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, hidden=None):
        x = self.embed(x)
        out, hidden = self.lstm(x, hidden)
        out = self.fc(out[:, -1, :])
        return out, hidden

# -------------------------------
# 4. Batch generator
# -------------------------------
def batch_generator(text_as_int, seq_length, batch_size):
    text_len = len(text_as_int)
    while True:
        X_batch = []
        y_batch = []
        for _ in range(batch_size):
            start_idx = np.random.randint(0, text_len - seq_length - 1)
            X_batch.append(text_as_int[start_idx:start_idx + seq_length])
            y_batch.append(text_as_int[start_idx + seq_length])
        X_batch = torch.tensor(np.array(X_batch, dtype=np.int64), dtype=torch.long).to(device)
        y_batch = torch.tensor(np.array(y_batch, dtype=np.int64), dtype=torch.long).to(device)
        yield X_batch, y_batch

# -------------------------------
# 5. Training setup
# -------------------------------
embed_size = 64
hidden_size = 256
num_layers = 2
seq_length = 50
batch_size = 128
num_epochs = 30
steps_per_epoch = 100
learning_rate = 0.005

model = CharLSTM(vocab_size, embed_size, hidden_size, num_layers).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

generator = batch_generator(text_as_int, seq_length, batch_size)

# -------------------------------
# 6. Train the model
# -------------------------------
for epoch in range(num_epochs):
    epoch_loss = 0
    for step in range(steps_per_epoch):
        X_batch, y_batch = next(generator)
        optimizer.zero_grad()
        output, _ = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss/steps_per_epoch:.4f}")

# -------------------------------
# 7. Optimized autocomplete function
# -------------------------------
def autocomplete(model, start_text, char2idx, idx2char, predict_len=100, temperature=0.5):
    model.eval()
    chars = [ch for ch in start_text]
    input_seq = torch.tensor([[char2idx[ch] for ch in chars]], dtype=torch.long).to(device)
    hidden = None

    for _ in range(predict_len):
        with torch.no_grad():
            output, hidden = model(input_seq, hidden)
            probs = F.softmax(output / temperature, dim=-1).squeeze()
            char_idx = torch.multinomial(probs, 1).item()
            chars.append(idx2char[char_idx])
            input_seq = torch.tensor([[char_idx]], dtype=torch.long).to(device)

    return ''.join(chars)

# -------------------------------
# 8. Interactive autocomplete loop
# -------------------------------
print("\nOptimized character-level LSTM autocomplete is ready! Type 'quit' to exit.\n")
while True:
    user_input = input("Type some text: ")
    if user_input.lower() == "quit":
        break
    if len(user_input) < 1:
        print("Type at least one character!")
        continue

    prediction = autocomplete(model, user_input[-seq_length:], char2idx, idx2char, predict_len=100, temperature=0.5)
    print("Autocomplete suggestion:\n", prediction)
    print()

Using device: cpu
Vocabulary size: 33
Epoch [1/30], Loss: 2.3705
Epoch [2/30], Loss: 1.8627
Epoch [3/30], Loss: 1.7076
Epoch [4/30], Loss: 1.6197
Epoch [5/30], Loss: 1.5563
Epoch [6/30], Loss: 1.5053
Epoch [7/30], Loss: 1.4315
Epoch [8/30], Loss: 1.4147
Epoch [9/30], Loss: 1.3778
Epoch [10/30], Loss: 1.3771
Epoch [11/30], Loss: 1.3584
Epoch [12/30], Loss: 1.3355
Epoch [13/30], Loss: 1.3169
Epoch [14/30], Loss: 1.3238
Epoch [15/30], Loss: 1.2814
Epoch [16/30], Loss: 1.2813
Epoch [17/30], Loss: 1.2672
Epoch [18/30], Loss: 1.2634
Epoch [19/30], Loss: 1.2324
Epoch [20/30], Loss: 1.2535
Epoch [21/30], Loss: 1.2521
Epoch [22/30], Loss: 1.2397
Epoch [23/30], Loss: 1.2391
Epoch [24/30], Loss: 1.2189
Epoch [25/30], Loss: 1.2006
Epoch [26/30], Loss: 1.2074
Epoch [27/30], Loss: 1.2013
Epoch [28/30], Loss: 1.2150
Epoch [29/30], Loss: 1.2090
Epoch [30/30], Loss: 1.1910

Optimized character-level LSTM autocomplete is ready! Type 'quit' to exit.

Type some text: hello world 
Autocomplete suggestion:
